# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [ ]:
%load_ext autoreload
%autoreload 2
import keras
import os, time
import pathlib
from usl_models.atmo_ml.model import AtmoModel
from usl_models.atmo_ml import dataset, visualizer, vars

import logging

logging.getLogger().setLevel(logging.INFO)

# climateiq-study-area-feature-chunks/NYC_Heat/NYC_summer_2000_01p
time_steps_per_day = 6
batch_size = 2

sim_dirs = [
    (
        "NYC_Heat_Test",
        [
            "NYC_summer_2000_01p",
            # 'NYC_summer_2010_99p',
            # 'NYC_summer_2015_50p',
            # 'NYC_summer_2017_25p',
            # 'NYC_summer_2018_75p'
        ],
    ),
    (
        "PHX_Heat_Test",
        [
            # 'PHX_summer_2008_25p',
            # 'PHX_summer_2009_50p',
            # 'PHX_summer_2011_99p',
            # 'PHX_summer_2015_75p',
            # 'PHX_summer_2020_01p'
        ],
    ),
]

sim_names = []
for sim_dir, subdirs in sim_dirs:
    for subdir in subdirs:
        sim_names.append(sim_dir + "/" + subdir)

print(sim_names)


output_vars = [
    vars.SpatiotemporalOutput.RH2
]
train_frac = 0.8

sim_name = "NYC_Heat_Test/NYC_summer_2000_01p"
date = "2000-05-25"

# Create training dataset with fused spatiotemporal data
train_ds = dataset.load_dataset_cached(
    pathlib.Path("data/filecache"),
    example_keys=[("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-25")],
    output_vars=output_vars,
).batch(batch_size=batch_size)
val_ds = dataset.load_dataset_cached(
    pathlib.Path("data/filecache"),
    example_keys=[("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-25")],
    output_vars=output_vars,
).batch(batch_size=batch_size)

In [ ]:
# Initialize the Atmo Model
model = AtmoModel(AtmoModel.Params(output_vars=output_vars))
model.summary(expand_nested=True)

In [ ]:
# Train the model
# Create a unique log directory by appending the current timestamp
log_dir = os.path.join("./logs", "run_" + time.strftime("%Y%m%d-%H%M%S"))
print(log_dir)
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
model.fit(train_ds, val_ds, epochs=50, callbacks=[tb_callback], validation_freq=10)

In [ ]:
visualizer.init_plt()
for input_batch, label_batch in val_ds.take(1):
    preds = model.call(input_batch)
    for b, _ in enumerate(label_batch):
        figs = visualizer.plot(
            inputs={k: v[b] for k, v in input_batch.items()},
            label=label_batch[b],
            pred=preds[b],
            st_var=vars.Spatiotemporal.RH,
            sto_var=vars.SpatiotemporalOutput.RH2,
            sim_name="test",
            date="date",
        )
        for fig in figs:
            fig.show()
        break